In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn import metrics
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error, r2_score

In [0]:
def evaluation_metrics(y_test, predictions, model_name=None):
    
    mae = mean_absolute_error(y_test, predictions)
    mdae = median_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)
    error_ratio = 100*mae/np.mean(y_test)
    
    print('====================================================')
    if model_name is not None:
        print('    Evaluation of', model_name, 'Model')
        print('****************************************************')
    # Evaluating the model using MAE Evaluation Metric
    print('Mean Absolute Error = ', mae)

    # Evaluating the model using Median Absolute Error Evaluation Metric
    print('Median Absolute Error = ', mdae)

    # Evaluating the model using MSE Evaluation Metric
    print('Mean Squared Error = ', mse)

    # Evaluating the model using RMSE Evaluation Metric
    print('Root Mean Squared Error = ', rmse)

    # Evaluating the model using R² Evaluation Metric
    print('R2 Score = ', r2)

    # Error percentage
    print('MAE to Mean Ratio Percentage=', error_ratio, '%')
    print('****************************************************')
    print('====================================================')

In [0]:
def dickey_fuller(ts, plot_log, print_output):
    
    '''
    Tests a time series for stationarity using the Dickey Fuller test.
    Inputs:
        - ts: time series for testing
        - plot_log: boolean to log transform the time series
        - print_output: boolean to print the results or not
    Returns:
        - Boolean for stationary or not
        - P value from the test
    '''
    
    if plot_log:
        dftest = adfuller(np.log(ts))
    else:
        dftest = adfuller(ts)

    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
        
    if dfoutput['p-value'] > 0.05:
        if print_output:
            print(f"Data is not stationary. P-value of {dfoutput['p-value']:.4f}")
        stationary = False
    else:
        if print_output:
            print(f"Data is stationary. P-value of {dfoutput['p-value']:.4f}")
        stationary = True
        
    return stationary, dfoutput['p-value']


In [0]:
def run_model(results, df_preds_, neighborhood, ts, order, seasonal_order, logged):
    
    '''
    This runs a SARIMAX model with the specified order for a given neighborhood
    '''
    
    train = ts[ts['future'] == 0]
    test = ts[ts['future'] == 1]

    
    if logged:
        sari_model = SARIMAX(train['ride_count_log'], order=order, seasonal_order=seasonal_order).fit(maxiter=1000, disp=False)
    else:
        sari_model = SARIMAX(train['ride_count'], order=order, seasonal_order=seasonal_order).fit(maxiter=1000, disp=False)
    
    # Add preds to predictions DataFrame
    preds = sari_model.forecast(steps = len(test))
    if logged:
        df_preds_.loc[:,neighborhood] = np.exp(preds)
    else:
        df_preds_.loc[:,neighborhood] = preds
    
    # Retrieve metrics
    if logged:
        model_results = report_metrics(test['ride_count_log'], preds, False)
    else:
        model_results = report_metrics(test['ride_count'], preds, False)
    
    # Calculate actual vs. predicted rides for 2021
    actual_rides_2021 = ts[ts.index > '12/31/2020']['ride_count'].sum()
    pred_rides_2021 = df_preds_[df_preds_.index > '12/31/2020'][neighborhood].sum()
    
    ride_delta = np.abs(pred_rides_2021 - actual_rides_2021)
    
    # Add results to results dataframe
    results.loc[neighborhood, 'model'] = sari_model
    results.loc[neighborhood, 'order'] = order
    results.loc[neighborhood, 'seasonal_order'] = seasonal_order
    results.loc[neighborhood, 'explained_variance'] = model_results[0]
    results.loc[neighborhood, 'MAE'] = model_results[1]
    results.loc[neighborhood, 'MSE'] = model_results[2]
    results.loc[neighborhood, 'R2'] = model_results[3]
    results.loc[neighborhood, '2021_actual'] = actual_rides_2021
    results.loc[neighborhood, '2021_predicted'] = pred_rides_2021
    results.loc[neighborhood, 'delta'] = ride_delta
    
    return results, df_preds_

In [0]:
def neighborhood_model_tune(pdq, seasonal_pdq, neighborhood, df, df_grid_search):
    best_ev = -1    
    train = df[df['future'] == 0]
    test = df[df['future'] == 1]
    for param in pdq:
        for param_seasonal in seasonal_pdq: 
            model = SARIMAX(train['ride_count_log'], order=param, seasonal_order=param_seasonal).fit(maxiter=1000, disp=False)
            y_pred = model.forecast(steps = len(test))
            ev = metrics.explained_variance_score(test['ride_count'], np.exp(y_pred))
            if ev > best_ev:
                best_ev = ev
                best_order = param
                best_s_order = param_seasonal
                df_grid_search.loc[neighborhood,:] = [best_order, best_s_order, best_ev]
    return df_grid_search